In [6]:
import requests
from bs4 import BeautifulSoup
import time
import unittest

In [7]:
def parse_object_page(url, object_n, object_list):
    """
    Fonction pour parser une page contenant des informations sur un type d'objet.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200 :

        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find('table', class_='ak-table')
        if table:
            for row in table.find_all('tr'):
                cells = row.find_all('td')
                if cells :
                    nom_index = 1
                    
                    if object_n in ["weapons", "equipment"]:
                        type_index = 3
                        niveau_index = 4
                    else:
                        type_index = 2
                        niveau_index = 3

                if object_n in ["weapons", "equipment", "consumables"] :
                    recipe = []
                    time.sleep(0.2)
                    try:
                        recipe_url = "https://www.dofus-touch.com" + cells[nom_index].find('a')['href']
                        recipe_response = requests.get(recipe_url, headers=headers)
                        recipe_soup = BeautifulSoup(recipe_response.text, "html.parser")
                        
                        # Here we find all elements with class 'ak-column' and 'ak-container'
                        recipe_elements = recipe_soup.find_all(class_='ak-column ak-container col-xs-12 col-sm-6')
                        
                        for element in recipe_elements:
                            # Find all <span> elements with class 'ak-linker'
                            span_elements = element.find_all('span', class_='ak-linker')
                            
                            # Loop through each <span> element and extract the text
                            for span in span_elements:
                                recipe.append(span.text.strip())
                    except :
                        continue 
                        
                    object_info = {
                        "name": cells[nom_index].text.strip(),
                        "type": cells[type_index].text.strip(),
                        "level": cells[niveau_index].text.strip(),
                        "recipe": recipe
                    }
                else :
                    object_info = {
                        "name": cells[nom_index].text.strip(),
                        "type": cells[type_index].text.strip(),
                        "level": cells[niveau_index].text.strip()
                    }

                object_list.append(object_info)
            return True  # Indique que la page a été analysée avec succès
    else:
        return False  # Indique qu'il y a eu une erreur lors de la requête

In [ ]:
base_url = "https://www.dofus-touch.com/en/mmorpg/encyclopedia"
objects = {
    "weapons": [],
    "equipment": [],
    "consumables": [],
    "resources": []
}

for object_n, object_list in objects.items():
    print(f"Object : {object_n}")
    url = f"{base_url}/{object_n}"
    parse_object_page(url, object_n, object_list)
    page = 2

    while True:
        page_url = f"{url}?page={page}"
        success = parse_object_page(page_url, object_n, object_list)
        if not success:
            break
        page += 1
    print(f"{len(object_list)} {object_n} find")

In [ ]:
import unittest

class TestParseObjectPage(unittest.TestCase):
    def setUp(self):
        # Setup initial object data
        self.test = {
            "weapons": [{"name": 'Riff Hatchet', "type": 'Axe', "level": 200, "recipe": []}],
            "equipment": [{"name": "Count Harebourg's Hat", "type": 'Axe', "level": 200, "recipe": ["Mahogany Pebble", "Leatherball Leather", "Sinistro Feather", "Glass Eye", "Harrogant Eye", "Armarine Blood", "Sinistrofu Wing", "Count Harebourg Essence"]}],
            "consumables": [{"name": 'Smoked Bearbarian Breast', "type": 'Edible meat', "level": 200, "recipe": ["Dose of Tasty Juice", "Frying Oil", "Spices", "Infected blood", "Cherry", "Onion", "Pepper Measure", "Preserved Bearbarian Breast"]}],
            "resources": [{"name": 'Celestial Bearbarian Breast', "type": 'Meat', "level": 200, "recipe": ["Preserved Celestial Bearbarian Breast"]}]
        }

        self.objects = {
            "weapons": [],
            "equipment": [],
            "consumables": [],
            "resources": []
        }
    def test_parse_weapons(self):
        # Test parsing of weapons
        success = parse_object_page(base_url, "weapons", self.objects["weapons"])
        self.assertFalse(success)
        self.assertEqual(self.objects["weapons"][0], self.test["weapons"][0])

    def test_parse_equipment(self):
        # Test parsing of equipment
        success = parse_object_page(base_url, "equipment", self.objects["equipment"])
        self.assertTrue(success)
        self.assertEqual(self.objects["equipment"][0], self.test["equipment"][0])

    def test_parse_consumables(self):
        # Test parsing of consumables
        success = parse_object_page(base_url, "consumables", self.objects["consumables"])
        self.assertTrue(success)
        self.assertEqual(self.objects["consumables"][0], self.test["consumables"][0])

    def test_parse_resources(self):
        # Test parsing of resources
        success = parse_object_page(base_url, "resources", self.objects["resources"])
        self.assertTrue(success)
        self.assertEqual(self.objects["resources"][0], self.test["resources"][0])

unittest.main()

In [ ]:
print("---------- test ----------")
print(f"weapons : {objects['weapons'][0]}")
print(f"equipment : {objects['equipment'][0]}")
print(f"consumables : {objects['consumables'][0]}")
print(f"resources : {objects['resources'][0]}")

In [ ]:
objects['weapons'][0] = {"name": 'Riff Hatchet', "type": 'Axe', "level": 200, "recipe": []}
objects['equipment'][0] = {"name": "Count Harebourg's Hat", "type": 'Axe', "level": 200, "recipe": ["Mahogany Pebble", "Leatherball Leather", "Sinistro Feather", "Glass Eye", "Harrogant Eye", "Armarine Blood", "Sinistrofu Wing", "Count Harebourg Essence"]}
objects['consumables'][0] = {"name": 'Smoked Bearbarian Breast', "type": 'Edible meat', "level": 200, "recipe": ["Dose of Tasty Juice", "Frying Oil", "Spices", "Infected blood", "Cherry", "Onion", "Pepper Measure", "Preserved Bearbarian Breast"]}
objects['resources'][0] = {"name": 'Celestial Bearbarian Breast', "type": 'Meat', "level": 200, "recipe": ["Preserved Celestial Bearbarian Breast"]}